# Compare simulation with experiment
________

Initiate laser pulse fron the phase-intensity image pairs of a wavefront censor (WFS) diagnostic:

ccd_name = 'ccd_pump_off.txt'
* This file contains the photon count, perhaps normalized, which we loosely refer to as intensity.

wfs_name = 'wfs_pump_off.txt'
* This file contains the phase of the electric field.
________

#### Imports

In [ ]:
import array
import math
import numpy as np
from pykern import pkcli
from pykern.pkcollections import PKDict
import os
from scipy.integrate import simpson

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse
from rslaser.optics import element
from rslaser.optics import drift
from rslaser.optics import crystal
from rslaser.utils import srwl_uti_data as srwutil

import scipy.constants as const
from scipy.ndimage import gaussian_filter
from scipy import special

import srwlib
from srwlib import srwl

# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

#### File data

In [ ]:
package_data_dir = rslaser.pkg_resources.resource_filename ('rslaser', 'package_data')

files = PKDict(
    meta = os.path.join(package_data_dir, 'wfs_meta.dat'),
    ccd = os.path.join(package_data_dir, 'ccd_pump_off.txt'),
    wfs = os.path.join(package_data_dir, 'wfs_pump_off.txt')
)

### Initialize Pulse

In [ ]:
params = PKDict(
    photon_e_ev = 1.5498, # Photon energy [eV], calculated from 800nm wavelength
    nslice      = 1,
    pulseE      = 1.0e-6,
    tau_fwhm    = 300.0e-12 / math.sqrt(2.),
    sigx_waist  = 1.2e-3 /1.18,
    sigy_waist  = 1.2e-3 /1.18,
)

thisPulse = pulse.LaserPulse(params, files)

## Plot Intensity From calc_int_from_wfr and combination of fields

In [ ]:
e_init_total = thisPulse.extract_total_2d_elec_fields()

x_init = np.linspace(thisPulse.slice[0].wfr.mesh.xStart, thisPulse.slice[0].wfr.mesh.xFin, thisPulse.slice[0].wfr.mesh.nx)
y_init = np.linspace(thisPulse.slice[0].wfr.mesh.yStart, thisPulse.slice[0].wfr.mesh.yFin, thisPulse.slice[0].wfr.mesh.ny)

intensity_init_total = 0.5 *const.c *const.epsilon_0 *(e_init_total.re**2.0 + e_init_total.im**2.0)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(6,3.6))
    ax = fig.gca()
    plt.pcolormesh(x_init*(1e3), y_init*(1e3), intensity_init_total, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (initial)')

In [ ]:
x_init = np.linspace(thisPulse.slice[0].wfr.mesh.xStart, thisPulse.slice[0].wfr.mesh.xFin, thisPulse.slice[0].wfr.mesh.nx)
y_init = np.linspace(thisPulse.slice[0].wfr.mesh.yStart, thisPulse.slice[0].wfr.mesh.yFin, thisPulse.slice[0].wfr.mesh.ny)

intensity_init_total = srwutil.calc_int_from_elec(thisPulse.slice[0].wfr)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(6,3.6))
    ax = fig.gca()
    plt.pcolormesh(x_init*(1e3), y_init*(1e3), intensity_init_total, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (initial)')

In [ ]:
max_num_l_slices = 50
max_intensity = np.zeros(max_num_l_slices)

for num_l_slices in np.arange(max_num_l_slices):

    #print('\n', num_l_slices+1, ' laser slices')
    params = PKDict(
        photon_e_ev = 1.5498, # Photon energy [eV], calculated from 800nm wavelength
        nslice      = num_l_slices+1,
        pulseE      = 1.0e-6,
        tau_fwhm    = 300.0e-12 / math.sqrt(2.),
        sigx_waist  = 1.2e-3 /1.18,
        sigy_waist  = 1.2e-3 /1.18,
    )

    thisPulse = pulse.LaserPulse(params, files)
    nslices_pulse = len(thisPulse.slice)
    
    e_total = thisPulse.extract_total_2d_elec_fields()
    intensity = 0.5 *const.c *const.epsilon_0 *(e_total.re**2.0 + e_total.im**2.0)
    max_intensity[num_l_slices] = np.max(intensity)

x_init = np.linspace(thisPulse.slice[0].wfr.mesh.xStart, thisPulse.slice[0].wfr.mesh.xFin, thisPulse.slice[0].wfr.mesh.nx)
y_init = np.linspace(thisPulse.slice[0].wfr.mesh.yStart, thisPulse.slice[0].wfr.mesh.yFin, thisPulse.slice[0].wfr.mesh.ny)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(6,3.6))
    ax = fig.gca()
    plt.pcolormesh(y_init*(1e3), x_init*(1e3), intensity, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (initial)')


n_l_slice = np.arange(1,max_num_l_slices + 1)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(n_l_slice,max_intensity,'k')
ax.tick_params(direction="in")
ax.set_ylabel(r'Maximum Intensity Value')
ax.set_xlabel(r'Number of Laser Slices in Pulse')